In [1]:
!pip install minepy
!pip install barplots
!pip install extra_keras_metrics
!pip install epigenomic_dataset
!pip install ucsc_genomes_downloader
!pip install keras_bed_sequence

     |████████████████████████████████| 495 kB 5.2 MB/s 
  Created wheel for minepy: filename=minepy-1.2.5-cp37-cp37m-linux_x86_64.whl size=177517 sha256=b7dd70ca7c686781e0d955de314633d16e617d58c295032fbd6a1a390d3d7b6d
  Stored in directory: /root/.cache/pip/wheels/d1/ea/d7/fabbfa6e294adcbc43dabca0e0158dafdd36051246992c7311
Successfully built minepy
  Created wheel for barplots: filename=barplots-1.1.11-py3-none-any.whl size=17877 sha256=adabb08e33012603ce1faba181d7046ab9d4146e5034142224e9ff1ba913f967
  Stored in directory: /root/.cache/pip/wheels/2a/53/e4/1c71fe6cb782f2f43b23872deea5369ad81c11118361558728
  Created wheel for sanitize-ml-labels: filename=sanitize_ml_labels-1.0.26-py3-none-any.whl size=7612 sha256=e07b7ed04bffe57221cdee4947459046b8ef1dfc1ec95fd8b4e55faaeb76864e
  Stored in directory: /root/.cache/pip/wheels/15/65/34/2e11a0da3cf6f112da5c402e01c454be619ff85a0fb38b3018
  Created wheel for compress-json: filename=compress_json-1.0.4-py3-none-any.whl size=4585 sha256=5dfc648

In [2]:
from epigenomic_dataset import active_enhancers_vs_inactive_enhancers
from epigenomic_dataset.utils import normalize_epigenomic_data
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from cache_decorator import Cache
from tqdm.keras import TqdmCallback
from barplots import barplots
from keras_mixed_sequence import MixedSequence, VectorSequence
from keras_bed_sequence import BedSequence
from ucsc_genomes_downloader import Genome

## Data retrieval
First, we retrieve the data and impute and scale them.

In [3]:
cell_line = "H1"
window_size = 256
task = "active_enhancers_vs_inactive_enhancers"

X, y = active_enhancers_vs_inactive_enhancers(
    cell_line=cell_line,
    window_size=window_size,
    binarize = True
)

genome = Genome("hg38")

bed_X = X.reset_index()
bed = bed_X[bed_X.columns[:5]]

Loading chromosomes for genome hg38:   0%|          | 0/25 [00:00<?, ?it/s]

In [4]:
#@title Build of the MMNN model by the best configuration of the FFNN and CNN models


In [5]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU, Concatenate, Layer
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D, GlobalMaxPool1D, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from extra_keras_metrics import get_complete_binary_metrics

In [6]:
from ast import literal_eval
best_configuration = pd.read_csv("best_models_enhancers_1.0_ffnn.csv")
best_configuration = literal_eval(best_configuration["1"][0])

In [7]:
op_learning_rate = best_configuration[0]
op_num_dense_layers = best_configuration[1]
op_num_of_nodes_3 = best_configuration[2]
op_num_of_nodes_2 = best_configuration[3]
op_num_of_nodes_1 = best_configuration[4]
op_l2_regularization = best_configuration[5]

In [8]:
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import AUC, Accuracy

def build_ffnn(X:pd.DataFrame):
    input_epigenomic_data = Input(shape=(X.shape[1]), name="epigenomic_data")
    hidden = input_epigenomic_data
    #Composition of the 3th layer
    if op_num_dense_layers == 3:
        name = 'layer_dense_3'
        hidden = Dense(op_num_of_nodes_2,
                        activation='relu',
                        kernel_regularizer=l2(op_l2_regularization), #I put the same regularization_factor 
                                                                         #to all the dense layers;
                        name=name)(hidden)
        
    #Composition of the 2th layer
    if op_num_dense_layers == 2:
        name = 'layer_dense_2'
        hidden = Dense(op_num_of_nodes_2,
                        activation='relu',
                        kernel_regularizer=l2(op_l2_regularization),
                        name=name)(hidden)
        
    #Composition of the 1th layer
    name = 'layer_dense_1'
    hidden = Dense(op_num_of_nodes_1,
                activation='relu',
                kernel_regularizer=l2(op_l2_regularization),
                name=name)(hidden)
    
    last_hidden_ffnn = hidden

    # use softmax-activation for classification.
    output_ffnn = Dense(1, activation='sigmoid')(hidden)
    
    ffnn = Model(
        inputs=input_epigenomic_data,
        outputs=output_ffnn,
        name="FFNN"
    )
    
    
    ffnn.compile(
        loss="binary_crossentropy",
        optimizer=Adam(learning_rate=op_learning_rate),
        metrics=get_standard_binary_metrics()
    )

    return ffnn, input_epigenomic_data, last_hidden_ffnn

In [9]:
from ast import literal_eval
best_configuration = pd.read_csv("best_models_enhancers_1.0_cnn.csv")
best_configuration = literal_eval(best_configuration["1"][0])

In [10]:
op_num_conv_units_2 = best_configuration[0]
op_num_of_nodes_2 = best_configuration[1]
op_num_of_nodes_1 = best_configuration[2]
op_kernel_size_2 = best_configuration[3]
op_dropout_prob = best_configuration[4]

In [11]:
from tensorflow.keras.layers import Dense, Input, Conv1D, Conv2D, Reshape, Flatten, MaxPool1D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from extra_keras_metrics import get_standard_binary_metrics

def build_cnn(window_size: int):
    input_sequence_data = Input(shape=(window_size, 4), name="sequence_data")
    hidden = Conv1D(64, kernel_size=5, activation="relu", padding="same")(input_sequence_data)
    hidden = BatchNormalization()(hidden)
    hidden = MaxPool1D()(hidden)
    hidden = Conv1D(64, kernel_size=5, activation="relu", padding="same")(input_sequence_data)
    hidden = BatchNormalization()(hidden)
    hidden = MaxPool1D()(hidden)
    hidden = Conv1D(64, kernel_size=5, activation="relu", padding="same")(input_sequence_data)
    hidden = BatchNormalization()(hidden)
    hidden = MaxPool1D()(hidden)
    hidden = Conv1D(op_num_conv_units_2, kernel_size=int(op_kernel_size_2), activation="relu", padding="same")(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = MaxPool1D()(hidden)
    hidden = Flatten()(hidden)
    hidden = Dense(op_num_of_nodes_2, activation="relu")(hidden)
    hidden = Dropout(op_dropout_prob)(hidden)
    hidden = Dense(op_num_of_nodes_1, activation="relu")(hidden)
    last_hidden_cnn = Dropout(op_dropout_prob)(hidden)
    
    output_cnn = Dense(1, activation="sigmoid")(last_hidden_cnn)
    
    cnn = Model(
        inputs=input_sequence_data,
        outputs=output_cnn,
        name="CNN"
    )
    
    cnn.compile(
        loss="binary_crossentropy",
        optimizer=Adam(op_learning_rate),
        metrics=get_standard_binary_metrics()
    )
    
    return cnn, input_sequence_data, last_hidden_cnn

In [12]:
def build_mmnn(
    X:pd.DataFrame=None,
    window_size:int=None,
    input_sequence_data:Layer=None,
    input_epigenomic_data:Layer=None,
    last_hidden_cnn:Layer=None,
    last_hidden_ffnn:Layer=None
):
    if X is not None:
      _, input_epigenomic_data, last_hidden_ffnn = build_ffnn(X)
    if window_size is not None:
      _, input_sequence_data, last_hidden_cnn = build_cnn(window_size)

    concatenation_layer = Concatenate()([
        last_hidden_ffnn,
        last_hidden_cnn
    ])

    last_hidden_mmnn = Dense(64, activation="relu")(concatenation_layer)
    output_mmnn = Dense(1, activation="sigmoid")(last_hidden_mmnn)

    mmnn = Model(
        inputs=[input_epigenomic_data, input_sequence_data],
        outputs=output_mmnn,
        name="MMNN"
    )

    mmnn.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )

    return mmnn

In [13]:
def get_cnn_sequence(
    genome:Genome,
    bed:pd.DataFrame,
    y:np.ndarray,
    batch_size=128
) -> MixedSequence:
    return MixedSequence(
        x={
            "sequence_data": BedSequence(
                genome,
                bed,
                batch_size=batch_size,
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )

def get_ffnn_sequence(
    X:np.ndarray,
    y:np.ndarray,
    batch_size=128
) -> MixedSequence:
    return MixedSequence(
        x={
            "epigenomic_data": VectorSequence(
                X,
                batch_size
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )

def get_mmnn_sequence(
    genome:Genome,
    bed:pd.DataFrame,
    X:np.ndarray,
    y:np.ndarray,
    batch_size=128
) -> MixedSequence:
    return MixedSequence(
        x={
            "sequence_data": BedSequence(
                genome,
                bed,
                batch_size=batch_size,
            ),
            "epigenomic_data": VectorSequence(
                X,
                batch_size
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )

### Evaluation of the models predictions
In order to evaluate the models predictions, we consider the Accuracy, AUPRC and AUROC metrics:

In [14]:
from tensorflow.keras.callbacks import EarlyStopping


def train_model(
    model:Model,
    model_name: str,
    task: str,
    training_sequence:MixedSequence,
    test_sequence:MixedSequence,
    holdout_number: int
):
    history = pd.DataFrame(model.fit(
        train_sequence,
        validation_data=test_sequence,
        epochs=100,
        verbose=False,
        callbacks=[
            EarlyStopping("val_loss", patience = 2),
            TqdmCallback(verbose=1)
        ]
    ).history)
    
    train_evaluation = dict(zip(model.metrics_names, model.evaluate(train_sequence, verbose=False)))
    test_evaluation = dict(zip(model.metrics_names, model.evaluate(test_sequence, verbose=False)))
    train_evaluation["run_type"] = "train"
    test_evaluation["run_type"] = "test"
    for evaluation in (train_evaluation, test_evaluation):
        evaluation["model_name"] = model_name
        evaluation["holdout_number"] = holdout_number
    
    evaluations = pd.DataFrame([
        train_evaluation,
        test_evaluation
    ])
    
    return history, evaluations

In [15]:
from sklearn.model_selection import StratifiedShuffleSplit

number_of_splits = 10

holdouts_generator = StratifiedShuffleSplit(
    n_splits=number_of_splits,
    test_size=0.2
)

In [16]:
from tensorflow.keras.callbacks import EarlyStopping
 
def train_model(
    model:Model,
    model_name: str,
    task: str,
    training_sequence:MixedSequence,
    test_sequence:MixedSequence,
    holdout_number: int
):
    history = pd.DataFrame(model.fit(
        train_sequence,
        validation_data=test_sequence,
        epochs=100,
        verbose=False,
        callbacks=[
            EarlyStopping("val_loss", patience = 2),
            TqdmCallback(verbose=1)
        ]
    ).history)
    
    train_evaluation = dict(zip(model.metrics_names, model.evaluate(train_sequence, verbose=False)))
    test_evaluation = dict(zip(model.metrics_names, model.evaluate(test_sequence, verbose=False)))
    train_evaluation["run_type"] = "train"
    test_evaluation["run_type"] = "test"
    for evaluation in (train_evaluation, test_evaluation):
        evaluation["model_name"] = model_name
        evaluation["holdout_number"] = holdout_number
    
    evaluations = pd.DataFrame([
        train_evaluation,
        test_evaluation
    ])
    
    return history, evaluations

In [17]:
# Create a list to store all the computed performance
all_performance = []

# Start the main loop, iterating through the holdouts
for holdout_number, (train_indices, test_indices) in tqdm(
    enumerate(holdouts_generator.split(X, y)),
    total=number_of_splits,
    desc="Computing holdouts"
):
    train_bed, test_bed = bed.iloc[train_indices], bed.iloc[test_indices]
    train_X, test_X = X.iloc[train_indices], X.iloc[test_indices]
    train_X, test_X = normalize_epigenomic_data(train_x=train_X,test_x=test_X)
    train_y, test_y = y.iloc[train_indices], y.iloc[test_indices]
    
    train_y = train_y.values.flatten()
    test_y = test_y.values.flatten()
    
    ffnn, input_epigenomic_data, last_hidden_ffnn = build_ffnn(train_X)
    cnn, input_sequence_data, last_hidden_cnn = build_cnn(window_size)
    model = build_mmnn(
        train_X, 
        window_size,
        input_sequence_data=input_sequence_data,
        input_epigenomic_data=input_epigenomic_data,
        last_hidden_ffnn=last_hidden_ffnn,
        last_hidden_cnn=last_hidden_cnn
    )
    
    train_sequence = get_mmnn_sequence(genome, train_bed, train_X, train_y)
    test_sequence = get_mmnn_sequence(genome, test_bed, test_X, test_y)

    # We compute the model performance
    history, performance = train_model(
        model,
        model.name,
        task,
        train_sequence,
        test_sequence,
        holdout_number
    )
    
    # We chain the computed performance to the performance list
    all_performance.append(performance)
        
# We convert the computed performance list into a DataFrame
all_performance = pd.concat(all_performance)

Computing holdouts:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric BalancedAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric Specificity implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric 

...

0batch [00:00, ?batch/s]

/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric BalancedAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric Specificity implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric 

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric BalancedAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric Specificity implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
/usr/local/lib/python3.7/dist-packages/keras/metrics.py:257: UserWarning: Metric 

In [24]:
all_performance.to_csv("all_performances_enhancers_1.0_mmnn.csv")

In [25]:
all_performance = pd.read_csv("all_performances_enhancers_1.0_mmnn.csv")

In [26]:
all_performance = all_performance.filter(items=['model_name', 'run_type', 'accuracy', 'AUROC', 'AUPRC'])
all_performance.groupby(['run_type', 'model_name']).mean()

,,accuracy,AUROC,AUPRC
run_type,model_name,,,
test,MMNN,0.989026,0.909927,0.340096
train,MMNN,0.991289,0.945615,0.564872


### Results visualization
Now that we have run our experiment we can visualize its performance:

In [27]:
barplots(
    all_performance,
    groupby=["model_name", "run_type"],
    orientation="horizontal",
    height=8
)

Rendering barplots:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/barplots/barplot.py:294: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  figure.tight_layout()
/usr/local/lib/python3.7/dist-packages/barplots/barplot.py:294: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  figure.tight_layout()
Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 470, in _handle_results
    task = get()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/figure.py", line 2

AssertionError: ignored

In [28]:
all_performance_ffnn = pd.read_csv("all_performances_enhancers_1.0_ffnn.csv")
all_performance_ffnn = all_performance_ffnn[all_performance_ffnn["use_feature_selection"] == False]
all_performance_ffnn = all_performance_ffnn.filter(items=['model_name', 'run_type', 'Accuracy', 'AUROC', 'AUPRC'])
all_performance_ffnn = all_performance_ffnn.rename(columns={'Accuracy': 'accuracy'})

all_performance_ccnn = pd.read_csv("all_performances_enhancers_1.0_cnn.csv")
all_performance_ccnn = all_performance_ccnn.filter(items=['model_name', 'run_type', 'accuracy', 'AUROC', 'AUPRC'])

In [29]:
all_performance = all_performance.append(all_performance_ffnn)
all_performance = all_performance.append(all_performance_ccnn)
all_performance

,model_name,run_type,accuracy,AUROC,AUPRC
0,MMNN,train,0.990401,0.935374,0.524456
1,MMNN,test,0.989018,0.900006,0.315056
2,MMNN,train,0.990756,0.951108,0.558622
3,MMNN,test,0.989097,0.911718,0.334959
4,MMNN,train,0.993916,0.969634,0.755577
5,MMNN,test,0.988860,0.886693,0.324876
6,MMNN,train,0.989255,0.931261,0.381726
7,MMNN,test,0.988860,0.919505,0.365511
8,MMNN,train,0.991704,0.952227,0.588700
9,MMNN,test,0.988781,0.902968,0.320799


In [30]:
from scipy.stats import wilcoxon

In [31]:
for outer_model in all_performance.model_name.unique():
    outer_model_performance = all_performance[
        (all_performance.model_name == outer_model) &
        (all_performance.run_type == "test")]
    for model in all_performance.model_name.unique():
        if outer_model >= model:
            continue
        model_performance = all_performance[
            (all_performance.model_name == model) &
            (all_performance.run_type == "test") 
        ]

        for metric in ("AUPRC", "AUROC", "accuracy"):
            outer, inner = outer_model_performance[metric], model_performance[metric]
            _, p_value = wilcoxon(outer, inner)
            if p_value < 0.01:
                if outer.mean() > inner.mean():
                    best_model, worse_model = outer_model, model
                else:
                    best_model, worse_model = model, outer_model
                print("The model {} outperforms the model {} with p-value {} on metric {}.".format(
                    best_model,
                    worse_model,
                    p_value,
                    metric
                ))
            else:
                print("The model {} is statistially indistinguishiable {} with p-value {} on metric {}.".format(
                    outer_model,
                    model,
                    p_value,
                    metric
                ))

The model MMNN outperforms the model FFNN with p-value 0.005062032126267864 on metric AUPRC.
The model FFNN is statistially indistinguishiable MMNN with p-value 0.1688069535565081 on metric AUROC.
The model FFNN is statistially indistinguishiable MMNN with p-value 0.016464232748736724 on metric accuracy.
The model MMNN outperforms the model CNN with p-value 0.005062032126267864 on metric AUPRC.
The model MMNN outperforms the model CNN with p-value 0.005062032126267864 on metric AUROC.
The model CNN is statistially indistinguishiable MMNN with p-value 0.0162542088976566 on metric accuracy.
The model FFNN outperforms the model CNN with p-value 0.005062032126267864 on metric AUPRC.
The model FFNN outperforms the model CNN with p-value 0.005062032126267864 on metric AUROC.
The model FFNN outperforms the model CNN with p-value 0.003580327522366853 on metric accuracy.
